Import libraries

In [3]:
import os
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import plotly.graph_objs as go
from IPython.display import Image

In [9]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Francesco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
wn_sysnets ={'Horse': wn.synset('horse.n.01'),
             'Cat': wn.synset('cat.n.01'),
             'Bird': wn.synset('bird.n.01'),
             'Cow': wn.synset('bovid.n.01'),
             'Dog': wn.synset('dog.n.01'),
             'Sheep': wn.synset('sheep.n.01'),
             'Aeroplane': wn.synset('airplane.n.01'),
             'Bicycle': wn.synset('bicycle.n.01'),
             'Bottle': wn.synset('bottle.n.01'),
             'Bus': wn.synset('bus.n.01'),
             'Car': wn.synset('car.n.01'),
             'Motorbike': wn.synset('motorcycle.n.01'),
             'Train': wn.synset('train.n.01')}

MAX_LVL = 5

**EVALUATION**

**Load the ROCAUC values and analyze them**

In [49]:
#@title
# load the ROCAUC curves results
with open('../VOC_vgg16_evaluation.json') as json_file: 
  eval_dict = json.load(json_file)

# intermediate dictionaries to record
holmes_auc_dict = {}
gradcam_auc_dict = {}
ratio_auc_dict = {}
pred_class_dict = {}
holonym_parts_dict = {}
num_pred_dict = {}

# other dictionaries
overall_holonym_auc_dict = {}

for i, holonym in enumerate(list(eval_dict.keys())):
  if holonym not in wn_sysnets:
    continue
  holonym_dict = eval_dict[holonym]      # holonym class dict
  holonym_parts = eval_dict[holonym]['parts']     # holonym parts
  holonym_parts_dict[holonym] = holonym_parts
  holonym_img_names = eval_dict[holonym]['stats'].keys() # VOC validation images for the holonym

  for j, img_name in enumerate(holonym_img_names):
    if j == 0:
        holmes_auc_dict[holonym] = []
        gradcam_auc_dict[holonym] = []
        ratio_auc_dict[holonym] = []
        pred_class_dict[holonym] = []
        num_pred_dict[holonym] = 0
    elif j>=50:
      break
    
    img_pred_class = holonym_dict['stats'][img_name]['pred_class']

    # list of pred values
    pred_class_dict[holonym].append(img_pred_class)
    # check if the hyponym prediction can be mapped to a holonym
    # first extract the predicted class synsets
    syns = wn.synsets(img_pred_class.replace(" ", "_"))
    #print("{} -> {}".format(img_pred_class, syns))
    # iterate on each synset
    found = 0
    for syn in syns:
        # look for the holonym as an hypernym up to a certain hierarchy level
        for lvl in range(MAX_LVL):
            if syn == wn_sysnets[holonym]:
                # found
                found = 1
                break
            else:
                # climb up the hierarchy
                hypernyms = syn.hypernyms()
                if not hypernyms:
                    break
                syn = hypernyms[0]    

        if found == 1:
            break
    if found == 0:
        continue
    #print("{} -> {}".format(img_pred_class, syn))
    
    num_pred_dict[holonym] += 1
    
    img_per_part_holmes_auc = holonym_dict['stats'][img_name]['HOLMES_AUCs']
    # list of list of aucs
    holmes_auc_dict[holonym].append(img_per_part_holmes_auc)
    
    img_per_part_gradcam_auc = holonym_dict['stats'][img_name]['GRADCAM_AUCs']
    # list of list of aucs
    gradcam_auc_dict[holonym].append(img_per_part_gradcam_auc)
    
    # list of list of auc ratios
    ratio_auc_dict[holonym].append([auc1 / auc2 for auc1, auc2 in zip(img_per_part_holmes_auc, img_per_part_gradcam_auc)])
    
  average_ratio_auc_per_holonym_meronym_dict = {k:np.mean(v, axis=0) for k,v in ratio_auc_dict.items()}

  average_ratio_auc_per_holonym_dict = {k:np.mean(v, axis=None) for k,v in ratio_auc_dict.items()}

In [50]:
num_pred_dict

{'Horse': 17,
 'Cat': 25,
 'Bird': 35,
 'Cow': 36,
 'Dog': 37,
 'Sheep': 25,
 'Aeroplane': 33,
 'Bicycle': 24,
 'Bottle': 4,
 'Bus': 33,
 'Car': 25,
 'Motorbike': 5,
 'Train': 2}

In [51]:
average_ratio_auc_per_holonym_dict

{'Horse': 1.3459849989649795,
 'Cat': 1.2243129860641888,
 'Bird': 1.9674253280841705,
 'Cow': 2.2680329850213488,
 'Dog': 1.5114268878905508,
 'Sheep': 1.4710175940705386,
 'Aeroplane': 1.1637459617875023,
 'Bicycle': 2.537180101491437,
 'Bottle': 1.759033788502818,
 'Bus': 1.565451334287713,
 'Car': 1.5759602312294412,
 'Motorbike': 1.979134913273834,
 'Train': 1.5146475233559862}

In [52]:
holonym_parts_dict

{'Horse': ['Head', 'Leg', 'Tail', 'Torso'],
 'Cat': ['Head', 'Leg', 'Tail', 'Torso'],
 'Bird': ['Head', 'Leg', 'Tail', 'Torso'],
 'Cow': ['Head', 'Horn', 'Leg', 'Torso'],
 'Dog': ['Head', 'Leg', 'Tail', 'Torso'],
 'Sheep': ['Head', 'Leg', 'Torso'],
 'Aeroplane': ['Artifact_Wing', 'Body', 'Engine', 'Stern', 'Wheel'],
 'Bicycle': ['Handlebar', 'Saddle', 'Wheel'],
 'Bottle': ['Body', 'Cap'],
 'Bus': ['Bodywork',
  'Door',
  'Headlight',
  'License_plate',
  'Mirror',
  'Wheel',
  'Window'],
 'Car': ['Bodywork',
  'Door',
  'Headlight',
  'License_plate',
  'Mirror',
  'Wheel',
  'Window'],
 'Motorbike': ['Handlebar', 'Headlight', 'Saddle', 'Wheel'],
 'Train': ['Coach', 'Headlight', 'Locomotive']}

In [53]:
average_ratio_auc_per_holonym_meronym_dict

{'Horse': array([1.59396154, 1.18991931, 1.3367582 , 1.26330094]),
 'Cat': array([1.17381455, 1.14980827, 1.51490689, 1.05872224]),
 'Bird': array([1.45129377, 1.35984265, 3.73757431, 1.32099058]),
 'Cow': array([2.82156465, 2.76609883, 2.2683349 , 1.21613356]),
 'Dog': array([1.85246702, 1.12085499, 1.35308719, 1.71929835]),
 'Sheep': array([2.04660997, 1.05535271, 1.3110901 ]),
 'Aeroplane': array([0.76148385, 1.13064959, 1.79473657, 1.4061344 , 0.7257254 ]),
 'Bicycle': array([2.41611383, 3.65398373, 1.54144275]),
 'Bottle': array([1.22617696, 2.29189062]),
 'Bus': array([1.33712582, 1.21010753, 1.21309762, 2.79299896, 1.13751841,
        2.13843714, 1.12887386]),
 'Car': array([1.24397746, 1.36177053, 1.93169188, 1.66555421, 1.35963651,
        1.85909661, 1.60999441]),
 'Motorbike': array([3.62125014, 1.56257886, 0.85464777, 1.87806288]),
 'Train': array([1.03401629, 2.16375918, 1.34616709])}

In [56]:
#set([holonym for holonym in pred_class_dict['Train'] if holonym not in holonym_mappings_dict['Train']])

In [ ]:
#pred_class_dict['Train']